In [58]:
# Install dependencies
!pip install conllu

In [88]:
# import headers
from conllu import parse,parse_incr
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
from tqdm import tqdm

In [60]:
with open ("data/UD_English-Atis/en_atis-ud-train.conllu", "r", encoding="utf-8") as f:
    data = f.read()
train_sentences = parse(data)
with open ("data/UD_English-Atis/en_atis-ud-test.conllu", "r", encoding="utf-8") as f:
    data = f.read()
dev_sentences = parse(data)
with open ("data/UD_English-Atis/en_atis-ud-test.conllu", "r", encoding="utf-8") as f:
    data = f.read()
test_sentences = parse(data)
print(train_sentences[0])

TokenList<what, is, the, cost, of, a, round, trip, flight, from, pittsburgh, to, atlanta, beginning, on, april, twenty, fifth, and, returning, on, may, sixth, metadata={sent_id: "0001.train", text: "what is the cost of a round trip flight from pittsburgh to atlanta beginning on april twenty fifth and returning on may sixth"}>


In [61]:
sentence = train_sentences[0]
sentence[1]

{'id': 2,
 'form': 'is',
 'lemma': 'be',
 'upos': 'AUX',
 'xpos': None,
 'feats': {'Mood': 'Ind',
  'Number': 'Sing',
  'Person': '3',
  'Tense': 'Pres',
  'VerbForm': 'Fin'},
 'head': 1,
 'deprel': 'cop',
 'deps': None,
 'misc': None}

In [62]:
def read_embeddings(filename, vocab_size=10000):
    with open(filename, encoding="utf-8") as file:
        word_embedding_dim = len(file.readline().split(" ")) - 1
    vocab = {}
    embeddings = np.zeros((vocab_size, word_embedding_dim))
    with open(filename, encoding="utf-8") as file:
        for idx, line in enumerate(file):
            if idx + 2 >= vocab_size:
                break
            cols = line.rstrip().split(" ")
            val = np.array(cols[1:])
            word = cols[0]
            embeddings[idx + 2] = val
            vocab[word] = idx + 2
    vocab["<UNK>"]=1
    vocab["<PAD>"]=0
  # a FloatTensor is a multidimensional matrix
  # that contains 32-bit floats in every entry
  # https://pytorch.org/docs/stable/tensors.html
    return torch.FloatTensor(embeddings), vocab

In [63]:
vocab_size = 50000
embeddings, vocab = read_embeddings('./data/glove.6B.100d.txt', vocab_size)

In [64]:
sentence[0]["form"].lower() in vocab

True

In [65]:
def build_vocab(sentences):
    data =[]
    word_set = set()
    vocab_dict={"<PAD>":0,"<UNK>":1}
    for sent in sentences:
        for token in sent:
            word_set.add(token["form"])
    word_list = sorted(list(word_set))
    for i,word in enumerate(word_list):
        vocab_dict[word]=i+2
    return vocab_dict
#vocab_dict=build_vocab(sentences)

In [66]:
def create_tags(sentences):
    tag_set=set()
    for sent in sentences:
        for token in sent:
            tag_set.add(token["upos"])
    tags=sorted(list(tag_set))
    tag_dict={"PAD":0}
    #tag_dict ={}
    for i,tag in enumerate(tags):
        tag_dict[tag]=i+1
    return tag_dict
    
tags = create_tags(train_sentences) 

In [67]:
tags

{'PAD': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'VERB': 13}

In [68]:
IGNORE_TAG_INDEX = 0
def create_data(sentences,vocab,tags,max_seq_len=50):
    sents_idx=[]
    sent_tags=[]
    #present=0
    #not_present=0
    for sent in sentences:
        sent_idx=[]
        sent_tag=[]
        for token in sent:
            if (token["form"].lower() in vocab):
                sent_idx.append(vocab[token["form"].lower()])
            else:
                sent_idx.append(vocab["<UNK>"])
            sent_tag.append(tags[token["upos"]])
        sents_idx.append(sent_idx)
        sent_tags.append(sent_tag)
    for i in range(len(sents_idx)):
        if len(sents_idx[i]) < max_seq_len:
            sents_idx[i]=sents_idx[i]+[vocab["<PAD>"] for _ in range(max_seq_len - len(sents_idx[i]))]
            sent_tags[i]=sent_tags[i]+[tags["PAD"] for _ in range(max_seq_len - len(sent_tags[i]))]
#     print(present)
#     print(not_present)
    return sents_idx,sent_tags

In [69]:
def sent_to_vector(sentence,max_seq_len=50):
    tokens = sentence.split(" ")
    sent_idx=[]
    for token in tokens:
        if (token.lower() in vocab):
            sent_idx.append(vocab[token.lower()])
        else:
            sent_idx.append(vocab["<UNK>"])
    for i in range(len(sent_idx)):
        if len(sent_idx) < max_seq_len:
            sent_idx=sent_idx+[vocab["<PAD>"] for _ in range(max_seq_len - len(sent_idx))]
    return sent_idx

In [70]:
sent_to_vector("Hi how are you")

[11085,
 199,
 34,
 83,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [71]:
x_train,y_train=create_data(train_sentences,vocab,tags,50)
x_dev,y_dev=create_data(dev_sentences,vocab,tags,50)
x_test,y_test=create_data(test_sentences,vocab,tags,50)
print("Train :"+str(len(x_train)) + " Dev : "+str(len(x_dev))+ " Test : "+str(len(x_test)))

Train :4274 Dev : 586 Test : 586


In [72]:
class POSDataSet(Dataset):
    def __init__(self, x, y):
        self.sent = torch.LongTensor(x)
        self.sent_tags = torch.LongTensor(y)
    
    def __getitem__(self, idx):
        return self.sent[idx], self.sent_tags[idx]
    
    def __len__(self):
        return len(self.sent)

In [73]:
train_dataset = POSDataSet(x_train,y_train)
dev_dataset = POSDataSet(x_dev,y_dev)
test_dataset = POSDataSet(x_test,y_test)

In [74]:
BATCH_SIZE =32
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [75]:
test_dataset.sent

tensor([[104,  34,   2,  ...,   0,   0,   0],
        [ 43, 305,   9,  ...,   0,   0,   0],
        [ 43, 410,   9,  ...,   0,   0,   0],
        ...,
        [ 43, 410,   9,  ...,   0,   0,   0],
        [ 43, 305,   6,  ...,   0,   0,   0],
        [ 54, 457, 287,  ...,   0,   0,   0]])

In [76]:
# The accuracy function has been implemented for you
def accuracy(true, pred):
  """
  Arguments:
  - true:       a list of true label values (integers)
  - pred:       a list of predicted label values (integers)

  Output:
  - accuracy:   the prediction accuracy
  """
  true = np.array(true)
  pred = np.array(pred)

  num_correct = sum(true == pred)
  num_total = len(true)
  return num_correct / num_total

In [81]:
def set_seed(seed):
  """
  Sets random seeds and sets model in deterministic
  training mode. Ensures reproducible results
  """
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(seed)

In [82]:
class POSTagger(nn.Module):
    def __init__(self,max_seq_len,embeddings,hidden_dim,n_layers,tagset_size):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.num_labels = tagset_size
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding.from_pretrained(embeddings,padding_idx=0)
        self.lstm = nn.LSTM(input_size=embeddings.size()[1], hidden_size= self.hidden_dim , num_layers=n_layers)
        self.hidden2tag = nn.Linear(self.hidden_dim,self.num_labels)
        
    def forward(self,input_seq):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_seq = input_seq.to(device)
        embed_out =self.embeddings(input_seq)
        lstm_out,_ = self.lstm(embed_out)
        logits = self.hidden2tag(lstm_out)
        return logits
    
    def evaluate(self,loader):
        self.eval()
        true_labels = []
        pred_labels = []
        for i, data in enumerate(loader):
            x,y = data
            logits = self.forward(x)
            pred_label=torch.argmax(logits, dim=-1).cpu().numpy()
            batch_size, _ = x.shape
            for j in range(batch_size):
                tags = y[j]
                pred = pred_label[j]
                for k in range(len(tags)):
                    if tags[k] != 0:
                        true_labels.append(tags[k])
                        pred_labels.append(pred[k])
        acc = accuracy(true_labels, pred_labels)  
        return acc ,true_labels ,pred_labels          

    def run_training(self,train_loader,dev_loader,epochs=100,learning_rate=5e-4,eval_every=5):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if str(device) == 'cpu':
            print("Training only supported in GPU environment")
            return
        torch.cuda.empty_cache()
        self.to(device)
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        loss_function = nn.CrossEntropyLoss(ignore_index=0)
        for epoch in tqdm(range(epochs)):
            self.train()
            total_loss = 0
            for i, data in enumerate(train_loader):
                x,y=data
                self.zero_grad()
                logits = self.forward(x)
                labels = torch.LongTensor(y).to(device)
                loss = loss_function(logits.view(-1, self.num_labels), labels.view(-1))
                total_loss += loss
                loss.backward()
                optimizer.step()
            # print("Epoch {} | Loss: {}".format(epoch, total_loss))
            # if epoch % eval_every == eval_every-1:
            #     acc,_,_ = self.evaluate(dev_loader)
            #     print("Epoch {} | Accuracy: {}".format(epoch, acc))
        acc_train,_,_ = self.evaluate(train_loader)
        acc_val,true_labels,pred_labels = self.evaluate(dev_loader)
        print("# Model : Training Accuracy : {} Validation Accuracy: {} #".format(acc_train,acc_val))  
    
    def predict(self,data):
        x = torch.LongTensor(data)
        self.eval()
        predictions = []
        logits = self.forward(x)
        pred_label=torch.argmax(logits, dim=-1).cpu().numpy()
        batch_size, _ = x.shape
        for j in range(batch_size):
            labels=[]
            for k in range(len(x[j])):
                if x[j][k] != 0:
                    labels.append(pred_label[j][k])
            predictions.append(labels)
        return predictions

    
    def save(self,path):
        torch.save(self.state_dict(), path)

    def load(self,path):
        self.load_state_dict(torch.load(path))

In [93]:
set_seed(159)
tagger = POSTagger(50,embeddings,128,2,len(tags))
tagger.run_training(train_dataloader,dev_dataloader,50,0.0005,5)

100%|██████████| 50/50 [00:32<00:00,  1.53it/s]


# Model : Training Accuracy : 0.9679580721405816 Validation Accuracy: 0.9673252279635258 #


In [94]:
tagger.save("pos_tagger.pt")

In [95]:
acc,_,_=tagger.evaluate(test_dataloader)
print(acc)

0.9673252279635258


In [83]:
def predictor(model : POSTagger,sentence,tags):
    tokens = sentence.split(" ")
    inv_map = {v: k for k, v in tags.items()}
    vec = [sent_to_vector(sentence)]
    predictions = model.predict(vec)
    for i in range(len(tokens)):
        print (tokens[i]+"\t"+inv_map[predictions[0][i]])

In [52]:
predictor(tagger,"Mary likes icecream",tags)

Mary	PROPN
likes	VERB
icecream	PROPN


Hyper Parameter Tuning and model exploration

In [86]:
def analyzer(train_dataloader,dev_dataloader,test_dataloader,tags):
    lrs = [0.0001,0.0005]
    layers = [1,2]
    hidden_dims = [32,64,128,256,512]
    best_accuracy = 0
    best_config = {"lr":0,"layers":0,"hidden_dim":0}
    for lr in lrs:
        for layer in layers:
            for hidden_dim in hidden_dims:
                set_seed(159)
                tagger = POSTagger(50,embeddings,hidden_dim,layer,len(tags))
                tagger.run_training(train_dataloader,dev_dataloader,50,lr,5)
                acc,true_labels,pred_labels=tagger.evaluate(test_dataloader)
                #score = f1_score(true_labels,pred_labels,len(tags))
                #cf = confusion_matrix(true_labels,pred_labels,len(tags))
                print("------------------------------------------------------------------")
                print("# Model Parameters | Learning Rate : {} Layers : {} Hidden Dim : {} #".format(lr,layer,hidden_dim))
                print("# Analysis | : Accuracy : {} #".format(acc))
                # print("# Confusion Matrix : # : ")
                # print(cf)
                print("------------------------------------------------------------------")
                if acc > best_accuracy:
                    tagger.save("pos_tagger.pt")
                    best_accuracy = acc
                    best_config = {"lr":lr,"layers":layer,"hidden_dim":hidden_dim}
                
    print("Best Accuracy : {} Best Config : {}".format(best_accuracy,best_config))

In [87]:
analyzer(train_dataloader,dev_dataloader,test_dataloader,tags)

100%|██████████| 50/50 [00:21<00:00,  2.38it/s]


# Model : Training Accuracy : 0.9444250333984174 Validation Accuracy: 0.9430091185410334 #
------------------------------------------------------------------
# Model Parameters | Learning Rate : 0.0001 Layers : 1 Hidden Dim : 32 #
# Analysis | : Accuracy : 0.9430091185410334 #
------------------------------------------------------------------


100%|██████████| 50/50 [00:21<00:00,  2.37it/s]


# Model : Training Accuracy : 0.9526872880485048 Validation Accuracy: 0.952887537993921 #


------------------------------------------------------------------
# Model Parameters | Learning Rate : 0.0001 Layers : 1 Hidden Dim : 64 #
# Analysis | : Accuracy : 0.952887537993921 #
------------------------------------------------------------------


...

100%|██████████| 50/50 [00:55<00:00,  1.11s/it]


# Model : Training Accuracy : 0.9685746583084986 Validation Accuracy: 0.9670212765957447 #


------------------------------------------------------------------
# Model Parameters | Learning Rate : 0.0005 Layers : 2 Hidden Dim : 256 #
# Analysis | : Accuracy : 0.9670212765957447 #
------------------------------------------------------------------


100%|██████████| 50/50 [01:11<00:00,  1.43s/it]


# Model : Training Accuracy : 0.9681841537354845 Validation Accuracy: 0.9656534954407295 #
------------------------------------------------------------------
# Model Parameters | Learning Rate : 0.0005 Layers : 2 Hidden Dim : 512 #
# Analysis | : Accuracy : 0.9656534954407295 #
------------------------------------------------------------------
Best Accuracy : 0.9673252279635258 Best Config : {'lr': 0.0005, 'layers': 2, 'hidden_dim': 128}


In [104]:
def tag_analysis(data_loader,model: POSTagger,tags):
    inv_map = {v: k for k, v in tags.items()}
    acc,true_labels,pred_labels=model.evaluate(data_loader)
    print(classification_report(true_labels,pred_labels,labels=list(tags.values()),target_names=list(tags.keys())))

In [105]:
tag_analysis(test_dataloader,tagger,tags)

              precision    recall  f1-score   support

         PAD       0.00      0.00      0.00         0
         ADJ       0.95      0.96      0.95       220
         ADP       0.97      1.00      0.98      1434
         ADV       0.91      0.79      0.85        76
         AUX       0.96      0.99      0.97       256
       CCONJ       1.00      1.00      1.00       109
         DET       0.99      0.86      0.92       512
        INTJ       0.95      1.00      0.97        36
        NOUN       0.99      0.98      0.98      1166
         NUM       0.94      0.95      0.95       127
        PART       0.84      0.29      0.43        56
        PRON       0.84      0.98      0.91       392
       PROPN       0.98      0.99      0.99      1567
        VERB       0.98      0.97      0.97       629

   micro avg       0.97      0.97      0.97      6580
   macro avg       0.88      0.84      0.85      6580
weighted avg       0.97      0.97      0.97      6580



/home/soumodiptab/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soumodiptab/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soumodiptab/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soumodiptab/.local/lib/pyth

In [110]:
class ImprovedPOSTagger(POSTagger):
    def __init__(self,max_seq_len,embeddings,hidden_dim,n_layers,tagset_size,dropout=0.8,bidirectional=True):
        super().__init__(max_seq_len,embeddings,hidden_dim,n_layers,tagset_size)
        self.lstm = nn.LSTM(input_size=embeddings.size()[1], hidden_size=hidden_dim, dropout=dropout, num_layers=n_layers, bidirectional=bidirectional)
        self.hidden2tag = nn.Linear(self.hidden_dim*2,self.num_labels)

In [111]:
tagger2 = ImprovedPOSTagger(50,embeddings,128,2,len(tags))
tagger2.run_training(train_dataloader,dev_dataloader,100,0.0005,5)

100%|██████████| 100/100 [01:37<00:00,  1.03it/s]


# Model : Training Accuracy : 0.9690884801150961 Validation Accuracy: 0.9679331306990882 #


In [112]:
tagger2.save('pos_tagger2.pt')

In [114]:
acc,_,_=tagger2.evaluate(test_dataloader)
print("Test accuracy for improved model: "+str(acc))

Test accuracy for improved model: 0.9679331306990882
